In [ ]:
# 6. 실행 파일 1 (최적화 비중 뽑아내는 것 까지)

import pandas as pd
import numpy as np
import config
from data_manager import DataManager
from portfolio_optimizer import PortfolioOptimizer
from backtester import DynamicBacktester
from visualizer import Visualizer
from utils import format_model_name

# --- 1. 데이터 준비 ---
dm = DataManager()
stock_data = dm.get_data(config.STOCKS, config.START_DATE, config.END_DATE, save_path=config.RAW_DATA_PATH)

# --- 2. 전략별 백테스팅 실행 ---
portfolio_results = {}

def strategy_wrapper(prices, **params):
    """최적화 로직을 백테스터에 전달하기 위한 래퍼 함수"""
    returns = dm.calculate_returns(prices, 'daily')
    optimizer = PortfolioOptimizer(returns)
    
    # Risk Parity는 별도 처리
    if params.get('is_optimizer') is False:
        return optimizer.get_risk_parity_weights(returns)
        
    # 나머지 최적화 전략
    num_assets = len(prices.columns)
    bounds = tuple(((config.WEIGHT_CONSTRAINTS['min'], config.WEIGHT_CONSTRAINTS['max'])) for _ in range(num_assets))
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
    target_return = params.get('target_return')
    
    if target_return is not None:
        constraints.append({'type': 'ineq', 'fun': lambda w: optimizer.calculate_metrics(w)['return'] - target_return})
        
    return optimizer.run_optimization(params['objective'], constraints=constraints, bounds=bounds)

# 활성화된 전략 실행
for name, params in config.STRATEGIES.items():
    if not params.get('enabled', False):
        continue
    
    model_name = format_model_name(name)
    print(f"\n--- '{model_name}' 모델 동적 백테스팅 시작 ---")
    
    backtester = DynamicBacktester(
        prices=stock_data,
        strategy_func=strategy_wrapper,
        strategy_params=params,
        initial_investment=config.INITIAL_INVESTMENT_USD,
        window=config.ROLLING_WINDOW_YEARS,
        frequency=config.REBALANCE_FREQUENCY,
        slippage=config.SLIPPAGE_PCT
    )
    portfolio_value = backtester.run()
    
    performance = DynamicBacktester.calculate_performance_metrics(portfolio_value, config.RISK_FREE_RATE_ANNUAL)
    portfolio_results[model_name] = {'value': portfolio_value, **performance}

# --- 3. 벤치마크 성과 계산 ---
benchmarks = {'SPY': 'SPY', 'BTC-USD': 'BTC-USD'}
for bm_name, bm_ticker in benchmarks.items():
    if bm_ticker in stock_data.columns:
        bm_price_data = stock_data[[bm_ticker]].dropna()
        bm_returns = bm_price_data.pct_change().dropna()
        portfolio_value = (config.INITIAL_INVESTMENT_USD * (1 + bm_returns).cumprod()).iloc[:, 0]
        performance = DynamicBacktester.calculate_performance_metrics(portfolio_value, config.RISK_FREE_RATE_ANNUAL)
        portfolio_results[f"Benchmark ({bm_name})"] = {'value': portfolio_value, **performance}

# --- 4. 최종 결과 요약 및 저장 ---
if portfolio_results:
    report_data = []
    print("\n\n--- 최종 성과 요약 ---")
    for name, metrics in portfolio_results.items():
        print(f"  - {name}: CAGR={metrics['CAGR']:.2%}, MDD={metrics['MDD']:.2%}, Sharpe={metrics['Sharpe Ratio']:.2f}, Calmar={metrics['Calmar Ratio']:.2f}")
        row = {k: v for k, v in metrics.items() if k not in ['value', 'drawdown_series']}
        row['Model'] = name
        report_data.append(row)
        
    report_df = pd.DataFrame(report_data).set_index('Model')
    
    # 저장 경로의 디렉터리가 없으면 생성
    save_path = config.PERFORMANCE_REPORT_SAVE_PATH
    save_path.parent.mkdir(parents=True, exist_ok=True)
    
    report_df.to_csv(save_path)
    print(f"\n상세 성과 보고서가 저장되었습니다: {save_path}")
    display(report_df)

    # --- 5. 시각화 ---
    visualizer = Visualizer(portfolio_results)
    visualizer.plot_performance_comparison()


'['SPY', 'QQQ', 'EFA', 'EEM', 'SSO', 'TQQQ', 'TLT', 'IEF', 'SHY', 'GLD', 'DBC', 'BTC-USD', 'ETH-USD', 'XRP-USD']' 종목 데이터 다운로드 시도 중...


[*********************100%***********************]  14 of 14 completed
/Users/012oov/Documents/Quant/MPT_Back/Source/backtester.py:26: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  rebal_dates = pd.to_datetime(self.prices.resample(self.frequency).last().index)


수정 종가 데이터가 로컬에 저장되었습니다: /Users/012oov/Documents/Quant/MPT_Back/Data/Raw data/downloaded_stock_prices.csv

--- 'Max Calmar Ratio' 모델 동적 백테스팅 시작 ---
2018-12-31: 가중치 재계산 중...
2019-12-31: 가중치 재계산 중...


/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/Users/012oov/Documents/Quant/MPT_Back/venv/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values 

2020-12-31: 가중치 재계산 중...


...

/Users/012oov/Documents/Quant/MPT_Back/Source/backtester.py:26: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  rebal_dates = pd.to_datetime(self.prices.resample(self.frequency).last().index)


2020-12-31: 가중치 재계산 중...
2021-12-31: 가중치 재계산 중...
2024-12-31: 가중치 재계산 중...

--- 'Target Return (return%)' 모델 동적 백테스팅 시작 ---
2018-12-31: 가중치 재계산 중...


/Users/012oov/Documents/Quant/MPT_Back/Source/backtester.py:26: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  rebal_dates = pd.to_datetime(self.prices.resample(self.frequency).last().index)


2019-12-31: 가중치 재계산 중...
2020-12-31: 가중치 재계산 중...
2021-12-31: 가중치 재계산 중...
2024-12-31: 가중치 재계산 중...


--- 최종 성과 요약 ---
  - Max Calmar Ratio: CAGR=12.08%, MDD=-56.45%, Sharpe=0.45, Calmar=0.21
  - Max Sharpe Ratio: CAGR=5.49%, MDD=-22.99%, Sharpe=0.50, Calmar=0.24
  - Risk Parity: CAGR=6.16%, MDD=-13.32%, Sharpe=1.15, Calmar=0.46
  - Minimum Variance: CAGR=6.01%, MDD=-16.70%, Sharpe=0.96, Calmar=0.36
  - Target Return (return%): CAGR=12.59%, MDD=-22.65%, Sharpe=0.97, Calmar=0.56
  - Benchmark (SPY): CAGR=14.22%, MDD=-33.72%, Sharpe=0.72, Calmar=0.42
  - Benchmark (BTC-USD): CAGR=45.50%, MDD=-83.04%, Sharpe=0.67, Calmar=0.55


OSError: Cannot save file into a non-existent directory: '/Users/012oov/Documents/Quant/MPT_Back/Results'